# XGBoost による顧客のチャーン予測
_**勾配ブースティング木を使ったモバイル顧客の離脱率予測**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Relative cost of errors](#Relative-cost-of-errors)
1. [Extensions](#Extensions)

---

## Background

_この Notebook は [AWS blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/) とそれに付随する SageMaker Examples の和訳です_

顧客を失うことはビジネスでは高く付きます。満足度の低い顧客を早い段階で特定することで、利用継続のインセンティブを与えられる可能性があります。この Notebook は機械学習 (ML) を用いて満足度の低い顧客を自動的に特定する方法 -- 顧客のチャーン予測 (customer churn prediction) とも呼ばれます -- を説明します。ML モデルが完璧な予測をすることはまれなので、この Notebook では ML を利用する経済的な結果を決める際の予測ミスの相対的なコストをどう取り入れるかについても書きます。

ここでは我々にとって身近なチャーン、携帯電話事業者を解約する例を用いることにします (不満ならいつでも見つけられそうです。) もし通信会社が自分が解約しようとしていることを知っているなら、一時的なインセンティブ -- いつでも携帯をアップグレードできるとか、新しい機能が使えるようになるとか -- を与えて契約を継続させるでしょう。インセンティブは通常、失った顧客を再獲得するよりも圧倒的にコスト効率が良いのです。

---

## Setup

_このノートブックの作成およびテストは ml.m4.xlarge ノートブックインスタンスを用いて行われました。_

以下のものを用意して始めましょう: 
- トレーニングとモデルデータの置き場所として使う S3 バケットとプレフィックス。ノートブックインスタンス・学習・デプロイ場所と同じリージョンにある必要があります。
- 学習・デプロイ用のコンテナがデータにアクセスするための IAM ロール ARN。これらの作成方法については[ドキュメント](https://docs.aws.amazon.com/ja_jp/IAM/latest/UserGuide/id_roles_create_for-service.html)を参照して下さい。_[注意] もしノートブックインスタンス、学習、デプロイの際に2つ以上のロールが必要な場合、boto regexp を適切な IAM ロール の ARN 文字列で置き換えて下さい。_

In [ ]:
# Define IAM role
import boto3
import re
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'DEMO-xgboost-churn'

次に、必要な Python ライブラリを import します。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

---
## Data

携帯電話事業者はどの顧客が解約し、誰がサービスを継続利用しているかの履歴データを持っています。この過去の情報を使って ML モデルを学習させることにより、ある事業者の解約率を予測するモデルを構築することができます。モデルを学習させた後、任意の顧客情報 (学習に使った情報と同様のもの) をモデルに入力することによって、顧客が解約しようとしているかを予測することができます。もちろん、モデルが予測を間違えることもあります -- 結局のところ、未来を予測することは一筋縄ではいかないということなのです。しかし、ここではその予測誤差とどう付き合っていくかについても言及します。

今回扱うのは、Daniel T. Larose の [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) という本で述べられている公開データセットです。これは University of California Irvine (UCI) 機械学習レポジトリの著者に帰属します。さて、ダウンロードしてデータセットを見てみましょう: 

In [ ]:
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip

In [ ]:
churn = pd.read_csv('./Data sets/churn.txt')
pd.set_option('display.max_columns', 500)
churn

最近の基準からすると比較的小さなデータセットで、とある US の携帯事業者の顧客データを 3,333 レコードと、それぞれ 21 個の属性で表しています。それぞれの属性は: 
- `State`: 顧客が住んでる US の州 (2文字の略号): OH, NJ など
- `Account Length`: このアカウントがアクティブだった日数
- `Area Code`: 顧客の電話番号に対応する3桁の地域コード
- `Phone`: のこりの7桁の電話番号
- `Int’l Plan`: 顧客が国際電話プランに契約しているかどうか: yes/no
- `VMail Plan`: 顧客がボイスメール機能を使っているかどうか: yes/no
- `VMail Message`: (恐らく) 月ごとのボイスメールの平均メッセージ数
- `Day Mins`: 日中に使用された通話時間の合計数
- `Day Calls`: 日中にかけられた電話の回数
- `Day Charge`: 昼間の通話料金
- `Eve Mins, Eve Calls, Eve Charge`: 夕方の通話料金
- `Night Mins`, `Night Calls`, `Night Charge`: 夜間の通話料金
- `Intl Mins`, `Intl Calls`, `Intl Charge`: 国際電話の通話料金
- `CustServ Calls`: カスタマーサービスへの通話数
- `Churn?`: 顧客が解約したかどうか: true/false

最後の属性 `Churn?` はターゲット属性として知られています -- 我々が ML モデルに予測してほしい属性です。ターゲット属性が2値なので、我々のモデルは2値の予測 (2値分類) を行うよう設計します。

それではデータを見てみましょう: 

In [ ]:
# Frequency tables for each categorical feature
for column in churn.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=churn[column], columns='% observations', normalize='columns'))

# Histograms for each numeric features
display(churn.describe())
%matplotlib inline
hist = churn.hist(bins=30, sharey=True, figsize=(10, 10))

すぐに以下のことが分かります: 
- `State` は一様に分布している。
- `Phone` は実用的でない大量の一意な値を取る。プレフィックスを使うことはできるかもしれないが、どういう割当がされてるか分からないなら使わないほうがよさそう。
- 顧客の 14% だけが解約している。2クラス間のデータ数に不均衡があるものの、それほど極端ではない。
- ほとんどの数値特徴量は驚くほどいい感じに分布していて、釣り鐘型 -- ガウシアン的な分布をしている。`VMail Message` は特筆すべき例外 (そして `Area Code` は非数値型に変換すべき特徴量として現れている)。

In [ ]:
churn = churn.drop('Phone', axis=1)
churn['Area Code'] = churn['Area Code'].astype(object)

次はそれぞれの特徴量とターゲット変数間の関係を見てみましょう。

In [ ]:
for column in churn.select_dtypes(include=['object']).columns:
    if column != 'Churn?':
        display(pd.crosstab(index=churn[column], columns=churn['Churn?'], normalize='columns'))

for column in churn.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = churn[[column, 'Churn?']].hist(by='Churn?', bins=30)
    plt.show()

面白いことにチャーンは: 
- 地理的に均等に分布している
- 国際プランに入っている傾向がある
- ボイスメールには入らない傾向にある
- 日毎の利用時間が二峰性になっている (非解約者に比べて高いか低いかのどちらか)
- 多くの顧客がカスタマーサービスを使っている (多くの問題を経験した顧客が解約しやすい、というのは理解できる)

ように見えます。これに加えて、解約者は `Day Mins` や `Day Charge` のような特徴量について非常に似通った分布をしていることがわかります。通話時間は金額と相関するため特に驚きはないですね。それでは特徴量の関係についてもう少し深く見てみましょう。

In [ ]:
display(churn.corr())
pd.plotting.scatter_matrix(churn, figsize=(12, 12))
plt.show()

いくつかの特徴量は本質的に 100% の相関を持っていることが分かります。これらの特徴量ペアを含め、些細な冗長性やバイアスとして扱われるものもありますが、いくつかの機械学習のアルゴリズムには致命的な問題となる可能性があります。それぞれから高い相関を持つペアを取り除いてみましょう。`Day Mins` から `Day Charge` を、`Night Mins` から `Night Charge` を、`Intl Mins` から `Intl Charge` を除外します:

In [ ]:
churn = churn.drop(['Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge'], axis=1)

さて、これでデータセットの前処理が終わったので、どのアルゴリズムを使うかを決定しましょう。上で述べたとおり、特定の値が (中間ではなく) 高いか低いかのどちらかにあればチャーンと予測できそうなことが分かっています。これを線形回帰などのアルゴリズムに入れるためには、多項式の (bucketed) 項を作る必要があります。かわりにこの問題を勾配ブースティング木を使ってモデル化することを考えましょう。Amazon SageMaker はマネージドで、分散学習でき、リアルタイム推論エンドポイントをデプロイできる XGBoost コンテナを提供しています。XGBoost は、特徴量とターゲット変数間の非線形な関係を自然に説明し、特徴量間の複雑な関係も記述する勾配ブースティング木 (gradient tree boosting) を用いています [[KDD'16](https://dl.acm.org/citation.cfm?id=2939785)]。

Amazon SageMaker XGBoost は CSV あるいは LibSVM フォーマットどちらのデータでも学習できます。この例では CSV にこだわります。データは
- 予測変数を1列目にもつ
- ヘッダ行をもたない

必要があります。まずはじめにカテゴリ変数を数値に変換しましょう。

In [ ]:
model_data = pd.get_dummies(churn)
model_data = pd.concat([model_data['Churn?_True.'], model_data.drop(['Churn?_False.', 'Churn?_True.'], axis=1)], axis=1)

そして、データを training, validation, test セットに分けましょう。これによりモデルの過学習を防ぐことができ、未知のデータを用いてモデルの精度をテストすることが可能になります。

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

これらのファイルを S3 にアップロードします。

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---
## Train

トレーニングに移ります。まず XGBoost アルゴリズムコンテナの場所を指定する必要があります。

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')

次に、今回 CSV ファイルを使ってトレーニングを行うので、S3 に置いたファイルのポインターとして学習用の関数が使う `s3_input` を作ります。

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

ここで、いくつかのパラメータ -- 学習に使うインスタンスの種類・数と XGBoost のハイパーパラメータ -- を指定します。いくつかの重要なパラメータは: 
- `max_depth` はアルゴリズム内でそれぞれの木がどれくらい深く作られるかをコントロールします。木が深いほどフィッティングは良くなりますが、計算量が多くなり過学習しやすくもなります。典型的にはモデルのパフォーマンスとトレードオフがあり、多数の浅い木と少数の深い木の間でパラメータを探索する必要があります。
- `subsample` は学習データのサンプリングをコントロールします。これは過学習を防ぎますが、あまり低くしすぎるとデータ不足になります。
- `num_round` はブースティングのラウンド数をコントロールします。これは本質的には前のイテレーションの残差を使って引き続きモデルを学習させます。これも、ラウンドを増やせば学習データに対するフィッティングは良くなりますが、計算量が増えたり過学習する可能性があります。
- `eta` は各ブースティングラウンドのアグレッシブさを決定します。値が大きい方が保守的なブースティングになります。
- `gamma` は木の成長がどれだけアグレッシブかを決めます。大きい値のほうが保守的なモデルを作ります。

詳細は GitHub [page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst) を読んで XGBoost のハイパーパラメータを確認して下さい。

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---
## Host

それではモデルを学習させたので、エンドポイントにデプロイしましょう。

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

### Evaluate

これでエンドポイントが立ち上がったので、http POST リクエストを投げることでリアルタイム推論を非常に簡単に行うことができます。しかしまず、`test_data` の NumPy array をエンドポイントの裏のモデルに渡すために、シリアライザーとデシリアライザーを設定しなければなりません。

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

以下の機能を持った簡単な関数を作ります: 
1. test データセットをループする
1. ミニバッチに分割する
1. CSV string payload に変換する
1. XGBoost エンドポイントを呼び出してミニバッチに対する予測を行う
1. 予測値を集めてモデルから出力された CSV を NumPy array に変換する

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.as_matrix()[:, 1:])

機械学習モデルのパフォーマンスを比較する多くの方法がありますが、単純に正しい値と予測値を比べてみましょう。この場合、単純に顧客が解約する (`1`) か、しない (`0`) かを予測することで、混同行列が得られます。

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])

_[注意] アルゴリズムの乱択性により、結果が少し違って見えるかもしれません。_

解約した人 48人のうち、39人を正しく予測することができました (true positive)。間違えて予測された4人はそのまま契約を続けるでしょう (false positive)。そのほか解約しないと予測された9人の顧客は実際に離脱しています (false negative)。

ここで重要な点は `np.round()` 関数が原因で0.5という単純な閾値 (カットオフ値) を設定していることです。`xgboost` からの予測は0から1の間の連続値を取るので、それを元々の2クラスに戻して解釈しています。しかし、離脱する顧客は、解約しようとして企業がより積極的に引き留めようとする顧客よりもコストがかかることが期待されるので、このカットオフを調整する必要があります。これはほとんど確実に false positive を増やしますが、同時にtrue positive も増やし、false negative を減らします。

ざっくりとした直観を得るために、予測結果の連続量を見てみましょう。

In [ ]:
plt.hist(predictions)
plt.show()

モデルから出力された連続値は0と1の間ですが、0.1と0.9の間に十分な値があるため、cutoffを変えれば実際に予測される顧客数が変化するはずです。例えば、

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.where(predictions > 0.3, 1, 0))

カットオフ値を0.5から0.3に変化させることで、もう1人の true positive と 3人の false positive、そして false negative が1人少なくなったことが分かります。数字は全体的に少ないですが、カットオフの変化により顧客の 6-10% に影響を与えています。これは正しい判断なのでしょうか？3人の顧客を繋ぎ止められますが、同時に5人に不要なインセンティブを与えていることになります。最適なカットオフを決めることは実世界に機械学習を応用する上で重要なステップになります。もう少し一般的に議論して、いくつかの仮定のもとでの解を考えましょう。

### Relative cost of errors

どんな2値分類問題も同じような感度のカットオフを生むことはありませんが、これ自体では特に問題とはなりません。結局、もし2クラスのスコアが十分簡単に分離可能なら、問題はそれほど難しくなく、ML ではなく単純なルールで問題を解くことが可能です。

これより重要なのはもし ML モデルで推論させても、モデルが間違って false positive と false negative を割り振るコストがあることです。同じように、true positives と true negatives の正しい推論に付随したコストも見る必要があります。なぜなら、カットオフの選び方はこれらの統計量の4つ全てに関わるからで、各推論に対して4つの出力がどのようなビジネス上の相対コストを生むか考慮する必要があるからです。

#### Assigning costs

今回問題にした携帯事業者の解約におけるコストとは何でしょうか? もちろんコストはビジネス上で取りうる具体的なアクションに依存します。ここでいくつかの仮定を置いてみましょう。

はじめに、true negative のコストを ¥0 (\*) とします。我々のモデルは本質的に、この場合幸せな顧客正しく特定できていることになるので、何もする必要はありません。

次に false negative は一番問題で、離脱しそうな顧客が留まると間違って予測するからです。顧客を失い、放棄所得、広告コスト、管理コスト、店頭コスト、そして恐らく携帯電話ハードウェア補助金を含む、代わりの顧客を獲得するコストを支払う必要があります。

最後に、解約しそうとモデルが予測した顧客については、リテンションのためのインセンティブが仮に一万円だとしましょう。もし事業会社が自分にこういう譲歩をしてきたら、さすがに解約までにもう一回考え直すでしょう。これが true positive と false positive の結果に対するコストです。ここで false posivie (顧客は満足だが誤って離脱と予測) の場合、一万円を _ドブに捨てる_ ことになります。恐らくこの一万円をもっと賢く使うことはできたでしょうが、既存の顧客の支持を高める可能性もあるため、それほど悪い選択ではありません。

\* 以下 \$1 = ¥100 というレートで訳すことにします

#### Finding the optimal cutoff

false negatives が false positives よりも相当コストがかかるのは明らかです。顧客数をもとにエラーを最適化するかわりに、このようなコスト関数を最小化しましょう: 

```txt
$500 * FN(C) + $0 * TN(C) + $100 * FP(C) + $100 * TP(C)
```

ここで `FN(C)` は false negative がカットオフ `C` の関数であることを意味し、`TN, FP, TP` についても同様です。上の式の結果を最小化するカットオフ `C` を探す必要があります。

これを行う素直な方法は、複数の考えられるカットオフについてシミュレーションを走らせることです。以下では100通りのカットオフについてforループで計算を行います。

In [ ]:
cutoffs = np.arange(0.01, 1, 0.01)
costs = []
for c in cutoffs:
    costs.append(np.sum(np.sum(np.array([[0, 100], [500, 100]]) * 
                               pd.crosstab(index=test_data.iloc[:, 0], 
                                           columns=np.where(predictions > c, 1, 0)))))

costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.show()
print('Cost is minimized near a cutoff of:', cutoffs[np.argmin(costs)], 'for a cost of:', np.min(costs))

上の図は、閾値を低くしすぎると全顧客に対してリテンションインセンティブを渡すことになりコストが跳ね上がることを示しています。一方で、閾値を高くしすぎるとあまりに多くの顧客を手放すことになり最終的に同じぐらいコストがかかります。全体のコストはカットオフが 0.46 のときに最小の 84万円 になり、何もしない場合に 200万円 以上失うのよりは圧倒的に良いという結果になります。

---
## Extensions

このノートブックでは顧客が離脱するのを予測するモデルを構築する方法と、true/false positive と false negative により生じるコストを最適化する閾値の決め方について披露しました。これを拡張するにはいくつかの方法が考えられます: 
- リテンションインセンティブを受け取る顧客も離脱の可能性がある。インセンティブを受け取っても解約する確率を含めるとリテンションプログラムのROIが向上する。
- 低価格のプランに移行したり課金オプションを無効化する顧客は別の種類のチャーンとしてモデル化できる。
- 顧客行動の発展をモデル化する。もし使用量が落ちてカスタマーサービスへの連絡回数が増えている場合、解約される可能性が高い。顧客情報は行動傾向を取り入れるべきである。
- 実際の学習データと金銭的コストはもっと複雑になり得る。
- それぞれのチャーンに合わせた複数のモデルが必要。

これらの複雑さに関わらず、このノートブックで説明したものと似たような原理は適用できるはずです。

### Optimizing model for prediction using Neo API
SageMaker Neo API を用いれば学習済みのモデルを特定のハードウェア用に最適化することが可能です。`compile_model()` 関数を呼ぶ際に、ターゲットとなるインスタンスファミリー (ここでは C5) とコンパイル済みのモデルを保存する S3 バケットを指定します。

** [重要] もし以下のコマンドが permission error になる場合、ノートブックの上部にスクロールして `get_execution_role()` により返される execution role の値を確認して下さい。このロールには ``output_path`` で指定される S3 バケットへのアクセス権限が必要です。

In [ ]:
output_path = '/'.join(xgb.output_path.split('/')[:-1])
compiled_model = xgb.compile_model(target_instance_family='ml_c5', 
                                   input_shape={'data':[1, 69]},
                                   role=role,
                                   framework='xgboost',
                                   framework_version='0.7',
                                   output_path=output_path)

### Creating an inference Endpoint

コンパイル済みのモデルをデプロイすることができます (コンパイルのターゲットと同じインスタンスを指定する必要があります)。この操作により、推論のための SageMaker エンドポイントが作成されます。

``deploy`` 関数の引数はエンドポイントに使われるインスタンス数とタイプを指定することができます。コンパイル時に指定されたインスタンスを選択して下さい (今回は `ml_c5` )。Neo API は Deep Learning Runtime (DLR) と呼ばれる特別なランタイムを使い最適化されたモデルを走らせます。

In [ ]:
# known issue: need to manually specify endpoint name
compiled_model.name = 'deployed-xgboost-customer-churn'
# There is a known issue where SageMaker SDK locates the incorrect docker image URI for XGBoost
# For now, we manually set Image URI
compiled_model.image = get_image_uri(sess.boto_region_name, 'xgboost-neo', repo_version='latest')
compiled_predictor = compiled_model.deploy(initial_instance_count = 1, instance_type = 'ml.c5.4xlarge')

### Making an inference request
コンパイルされたモデルは CSV の入力を受け付けます。

In [ ]:
compiled_predictor.content_type = 'text/csv'
compiled_predictor.serializer = csv_serializer
compiled_predictor.deserializer = None

In [ ]:
def optimized_predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, compiled_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

# Batch prediction is not supported yet; need to send one data point at a time
dtest = test_data.as_matrix()
predictions = []
for i in range(dtest.shape[0]):
    predictions.append(optimized_predict(dtest[i:i+1, 1:]))
predictions = np.array(predictions).squeeze()
predictions

### (Optional) Clean-up

このノートブックによって作られたリソースを削除していい場合、以下のセルを実行してください。このコマンドは上で作成したエンドポイントを削除して意図しない請求を防ぐことができます。
(必要であれば、このノートブック自体を走らせているノートブックインスタンスも SageMaker のマネージメントコンソールから停止させて下さい。)

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
sagemaker.Session().delete_endpoint(compiled_predictor.endpoint)